In [1]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [2]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [3]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [4]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [5]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [7]:
llm('can i still join the course ?')

" Certainly! However, please note that I cannot directly enroll you in courses or provide personalized advice. Generally speaking:\n\nTo determine if and how to join a specific online course like those offered by platforms such as Coursera (which seems to be your context based on this request), follow these steps:\n\n1. **Course Availability**: Check the availability of the course you are interested in joining, either through their website or an educational institution that partners with them. Some courses might have limited slots due to high demand and a fixed capacity for interactive sessions like those led by industry experts such as Elon Musk (if relevant).\n\n2. **Prerequisites**: Make sure you meet any prerequisites required for the course, which can often be found on the Course Description page or in the Syllabus section of the course website. These might include basic knowledge in a specific subject area and certain computer skills to handle interactive components effectively.\

In [8]:
print(_)

 Certainly! However, please note that I cannot directly enroll you in courses or provide personalized advice. Generally speaking:

To determine if and how to join a specific online course like those offered by platforms such as Coursera (which seems to be your context based on this request), follow these steps:

1. **Course Availability**: Check the availability of the course you are interested in joining, either through their website or an educational institution that partners with them. Some courses might have limited slots due to high demand and a fixed capacity for interactive sessions like those led by industry experts such as Elon Musk (if relevant).

2. **Prerequisites**: Make sure you meet any prerequisites required for the course, which can often be found on the Course Description page or in the Syllabus section of the course website. These might include basic knowledge in a specific subject area and certain computer skills to handle interactive components effectively.

3. **R